In [ ]:
import pandas as pd
import os
import fitz

In [ ]:
path = 'F:/Environmental Baseline Data/Version 4 - Final/PDF/'
pdfs = os.listdir(path)

list_df = []
for pdf in pdfs[:5]:
    try:
        dataID = pdf.split('.')[0]
        doc = fitz.open(path + pdf)
        page_count = doc.pageCount
        for page in doc:
            figures = page.searchFor('Figure')
            page_num = page.number
            rotation = page.rotation
            try:
                image_list = page.getImageList() # get list of used images
            except:
                image_list = []
                print('Error getting image list for:', pdf, page_num)
                pass
            num_images = len(image_list)
            text = page.getText('dict') # list, extract the page’s text
            width = text['width']
            height = text['height']
            media_x0 = page.MediaBox[0]
            media_y0 = page.MediaBox[1]
            media_x1 = page.MediaBox[2]
            media_y1 = page.MediaBox[3]
            media_width = page.MediaBoxSize[0]
            media_height = page.MediaBoxSize[1]

            for block in text['blocks']:
                t = block['type']
                bbox = block['bbox']
                if t == 1:
                    ext, color, xres, yres, bpc, image = block['ext'], block['colorspace'], block['xres'], block['yres'], block['bpc'], block['image']
                    block_width, block_height = block['width'], block['height']
                else:
                    ext, color, xres, yres, bpc, image = None, None, None, None, None, None
                    block_width, block_height = None, None
                list_df.append({'dataID':dataID, 'page_count':page_count, 'page_num':page_num,  'width':width, 'height':height, 
                                'rotation':rotation, 'figures':len(figures), 'num_images':num_images, 'type':t, 
                                'block_width':block_width, 'block_height':block_height, 
                                'media_x0':media_x0, 'media_y0':media_y0, 'media_x1':media_x1, 'media_y1':media_y1,
                                'media_width':media_width, 'media_height':media_height,
                                'bbox_x0':bbox[0], 'bbox_y0':bbox[1], 'bbox_x1':bbox[2], 'bbox_y1':bbox[3], 
                                'ext':ext, 'color':color, 'xres':xres, 'yres':yres, 'bpc':bpc})
    except:
        print('Error for id:', pdf)
        pass
df = pd.DataFrame(list_df)
df['page_area'] = df['width'] * df['height']
df['block_area'] = df['block_width'] * df['block_height']
df['bbox_width'] = df['bbox_x1'] - df['bbox_x0']
df['bbox_height'] = df['bbox_y1'] - df['bbox_y0']
df['bbox_area'] = df['bbox_width'] * df['bbox_height']
df['bbox_area_image'] = df['bbox_area'] * df['type']

# df to table in db
df.to_csv('image_test.csv', index=False)


In [ ]:
df.head()